In [6]:
%env HV_DOC_HTML = true

env: HV_DOC_HTML=true


In [7]:
pip install -q hvplot

In [8]:
pip install -q holoviews

In [9]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [10]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv('lending_data.csv')

# Review the DataFrame
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [11]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df['loan_status']

# Separate the X variable, the features
X = lending_data_df.drop(columns='loan_status')

In [12]:
# Review the y variable Series
y

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [13]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [14]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [15]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [16]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

# Fit the model using training data
model.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [17]:
# Make a prediction using the testing data
predictions = model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [18]:
# Print the balanced_accuracy score of the model
print(f'Balanced Accuracy Score: {balanced_accuracy_score(y_test, predictions)}')

Balanced Accuracy Score: 0.9520479254722232


In [19]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[18663,   102],
       [   56,   563]])

In [20]:
# Print the classification report for the model
print(classification_report(y_test, predictions, target_names=['healthy', 'high risk']))

              precision    recall  f1-score   support

     healthy       1.00      0.99      1.00     18765
   high risk       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Healthy loans are predicted almost perfectly. The model seems to overcorrect high risk loans, putting 15% of them in the 'healthy' category. This might be caused by a lower sample size in testing.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [21]:
# pip install for google colab
!pip install imblearn

In [25]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# Assign a random_state parameter of 1 to the model
rosModel = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = rosModel.fit_resample(X, y)

In [29]:
# Count the distinct values of the resampled labels data
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0, 75036), (1, 75036)]


In [42]:
list(Counter(y_resampled))

[0, 1]

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [44]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lrModel = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

# Fit the model using the resampled training data
lrModel.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
lrPredictions = lrModel.predict(X_resampled)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [47]:
# Print the balanced_accuracy score of the model
print(f'Balanced Accuracy Score: {balanced_accuracy_score(y_resampled, lrPredictions)}')

Balanced Accuracy Score: 0.9945026387334079


In [49]:
# Generate a confusion matrix for the model
confusion_matrix(y_resampled, lrPredictions)

array([[74614,   422],
       [  403, 74633]])

In [50]:
# Print the classification report for the model
print(classification_report(y_resampled, lrPredictions, target_names=['healthy', 'high risk']))

              precision    recall  f1-score   support

     healthy       0.99      0.99      0.99     75036
   high risk       0.99      0.99      0.99     75036

    accuracy                           0.99    150072
   macro avg       0.99      0.99      0.99    150072
weighted avg       0.99      0.99      0.99    150072



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicts loans in both categories almost perftectly. The increase in training samples has helped the predictions greatly. However, the precision score for healthy loans dropped by a percent.